In [34]:
import pandas as pd 
import numpy as np
import json, urllib.request
from datetime import datetime
from datetime import timedelta
from collections import Counter

transaction_data = pd.read_json('transaction-data-adhoc-analysis.json')


In [35]:
months_loop = [pd.to_datetime(x).strftime("%m") for x in list(transaction_data['transaction_date'])]
transaction_data.insert(2,"Months",months_loop, True)


In [36]:
table_1_1 = transaction_data.drop(["transaction_date","address","birthdate","mail","name","username","sex"], axis = 1, inplace=False)
table_1_1.head()

,Months,transaction_items,transaction_value
0,01,"Exotic Extras,Beef Chicharon,(x4)",5196
1,01,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056
2,01,"HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3...",5500
3,01,"HealthyKid 3+,Yummy Vegetables,(x1)",500
4,01,"Candy City,Orange Beans,(x3);Candy City,Gummy ...",2697


In [37]:
new_column = table_1_1["transaction_items"].str.split(";", expand = True)
new_column

,0,1,2
0,"Exotic Extras,Beef Chicharon,(x4)",None,None
1,"Exotic Extras,Beef Chicharon,(x3)","HealthyKid 3+,Nutrional Milk,(x4)","Candy City,Orange Beans,(x1)"
2,"HealthyKid 3+,Gummy Vitamins,(x3)","HealthyKid 3+,Yummy Vegetables,(x2)",None
3,"HealthyKid 3+,Yummy Vegetables,(x1)",None,None
4,"Candy City,Orange Beans,(x3)","Candy City,Gummy Worms,(x4)","HealthyKid 3+,Gummy Vitamins,(x1)"
...,...,...,...
83031,"Exotic Extras,Kimchi and Seaweed,(x2)","HealthyKid 3+,Nutrional Milk,(x1)",None
83032,"HealthyKid 3+,Yummy Vegetables,(x1)","Exotic Extras,Kimchi and Seaweed,(x1)","Candy City,Orange Beans,(x4)"
83033,"Candy City,Orange Beans,(x4)","Candy City,Gummy Worms,(x4)","HealthyKid 3+,Yummy Vegetables,(x4)"
83034,"HealthyKid 3+,Nutrional Milk,(x2)","Candy City,Gummy Worms,(x2)",None


In [38]:
table_1_1["First Product"]= new_column[0]

table_1_1["Second Product"]= new_column[1]

table_1_1["Third Product"]= new_column[2]
  
table_1_1.drop(columns =["transaction_items"], inplace = True)

table_1_1.head()

,Months,transaction_value,First Product,Second Product,Third Product
0,01,5196,"Exotic Extras,Beef Chicharon,(x4)",None,None
1,01,12056,"Exotic Extras,Beef Chicharon,(x3)","HealthyKid 3+,Nutrional Milk,(x4)","Candy City,Orange Beans,(x1)"
2,01,5500,"HealthyKid 3+,Gummy Vitamins,(x3)","HealthyKid 3+,Yummy Vegetables,(x2)",None
3,01,500,"HealthyKid 3+,Yummy Vegetables,(x1)",None,None
4,01,2697,"Candy City,Orange Beans,(x3)","Candy City,Gummy Worms,(x4)","HealthyKid 3+,Gummy Vitamins,(x1)"


In [39]:
first_product = table_1_1["First Product"].str.split(",", expand = True)
second_product = table_1_1["Second Product"].str.split(",", expand = True)
third_product = table_1_1["Third Product"].str.split(",", expand = True)

#First Product
table_1_1["First Product Brand"]= first_product[0]

table_1_1["First Product Item"]= first_product[1]

table_1_1["First Product Count"]= first_product[2]

#Second Product
table_1_1["Second Product Brand"]= second_product[0]

table_1_1["Second Product Item"]= second_product[1]

table_1_1["Second Product Count"]= second_product[2]

#Third Product
table_1_1["Third Product Brand"]= third_product[0]

table_1_1["Third Product Item"]= third_product[1]

table_1_1["Third Product Count"]= third_product[2]

#drop the original columns
table_1_1.drop(columns =["First Product"], inplace = True)
table_1_1.drop(columns =["Second Product"], inplace = True)
table_1_1.drop(columns =["Third Product"], inplace = True)

table_1_1.head()

,Months,transaction_value,First Product Brand,First Product Item,First Product Count,Second Product Brand,Second Product Item,Second Product Count,Third Product Brand,Third Product Item,Third Product Count
0,01,5196,Exotic Extras,Beef Chicharon,(x4),None,None,None,None,None,None
1,01,12056,Exotic Extras,Beef Chicharon,(x3),HealthyKid 3+,Nutrional Milk,(x4),Candy City,Orange Beans,(x1)
2,01,5500,HealthyKid 3+,Gummy Vitamins,(x3),HealthyKid 3+,Yummy Vegetables,(x2),None,None,None
3,01,500,HealthyKid 3+,Yummy Vegetables,(x1),None,None,None,None,None,None
4,01,2697,Candy City,Orange Beans,(x3),Candy City,Gummy Worms,(x4),HealthyKid 3+,Gummy Vitamins,(x1)


In [40]:
table_1_1.fillna(value=np.nan, inplace=True)
table_1_1["First Product Count"]=table_1_1["First Product Count"].apply(lambda x: str(x).replace('(','').replace('x','').replace(')',''))
table_1_1["Second Product Count"] = [str(x).replace('(','').replace('x','').replace(')','') for x in table_1_1["Second Product Count"]]
table_1_1["Third Product Count"] = [str(x).replace('(','').replace('x','').replace(')','') for x in table_1_1["Third Product Count"]]
table_1_1 = table_1_1.apply(lambda x: x.replace('nan','0'))

In [41]:
table_1_1["First Product Count"] = [int(x) for x in table_1_1["First Product Count"]]
table_1_1["Second Product Count"] = [int(x) for x in table_1_1["Second Product Count"]]
table_1_1["Third Product Count"] = [int(x) for x in table_1_1["Third Product Count"]]
table_1_1.head()

,Months,transaction_value,First Product Brand,First Product Item,First Product Count,Second Product Brand,Second Product Item,Second Product Count,Third Product Brand,Third Product Item,Third Product Count
0,01,5196,Exotic Extras,Beef Chicharon,4,NaN,NaN,0,NaN,NaN,0
1,01,12056,Exotic Extras,Beef Chicharon,3,HealthyKid 3+,Nutrional Milk,4,Candy City,Orange Beans,1
2,01,5500,HealthyKid 3+,Gummy Vitamins,3,HealthyKid 3+,Yummy Vegetables,2,NaN,NaN,0
3,01,500,HealthyKid 3+,Yummy Vegetables,1,NaN,NaN,0,NaN,NaN,0
4,01,2697,Candy City,Orange Beans,3,Candy City,Gummy Worms,4,HealthyKid 3+,Gummy Vitamins,1


In [42]:
table_1 = table_1_1
table_1["First Product Brand"] = table_1[["First Product Brand", "First Product Item"]].apply(" | ".join, axis=1)
table_1.drop(["First Product Item"], axis = 1, inplace=True)
table_1["Second Product Brand"] = table_1["Second Product Brand"].map(str) + " | " + table_1["Second Product Item"].map(str)
table_1.drop(["Second Product Item"], axis = 1, inplace=True)
table_1["Third Product Brand"] = table_1["Third Product Brand"].map(str) + " | " +table_1["Third Product Item"].map(str)
table_1.drop(["Third Product Item"], axis = 1, inplace=True)
table_1.head()

,Months,transaction_value,First Product Brand,First Product Count,Second Product Brand,Second Product Count,Third Product Brand,Third Product Count
0,01,5196,Exotic Extras | Beef Chicharon,4,nan | nan,0,nan | nan,0
1,01,12056,Exotic Extras | Beef Chicharon,3,HealthyKid 3+ | Nutrional Milk,4,Candy City | Orange Beans,1
2,01,5500,HealthyKid 3+ | Gummy Vitamins,3,HealthyKid 3+ | Yummy Vegetables,2,nan | nan,0
3,01,500,HealthyKid 3+ | Yummy Vegetables,1,nan | nan,0,nan | nan,0
4,01,2697,Candy City | Orange Beans,3,Candy City | Gummy Worms,4,HealthyKid 3+ | Gummy Vitamins,1


In [43]:
table_1 = table_1.apply(lambda x: x.replace('nan | nan','No Item'))
table_1.head(7)

,Months,transaction_value,First Product Brand,First Product Count,Second Product Brand,Second Product Count,Third Product Brand,Third Product Count
0,01,5196,Exotic Extras | Beef Chicharon,4,No Item,0,No Item,0
1,01,12056,Exotic Extras | Beef Chicharon,3,HealthyKid 3+ | Nutrional Milk,4,Candy City | Orange Beans,1
2,01,5500,HealthyKid 3+ | Gummy Vitamins,3,HealthyKid 3+ | Yummy Vegetables,2,No Item,0
3,01,500,HealthyKid 3+ | Yummy Vegetables,1,No Item,0,No Item,0
4,01,2697,Candy City | Orange Beans,3,Candy City | Gummy Worms,4,HealthyKid 3+ | Gummy Vitamins,1
5,01,2990,HealthyKid 3+ | Nutrional Milk,1,HealthyKid 3+ | Yummy Vegetables,2,No Item,0
6,01,3196,Exotic Extras | Kimchi and Seaweed,4,No Item,0,No Item,0


In [44]:
answer_table1 = pd.pivot_table(table_1, index = ['First Product Brand'],
                          columns = 'Months',
                          values = 'First Product Count',
                          aggfunc = ['sum'],
                          fill_value=None,
                          margins = False,
                          dropna = True,
                          margins_name = 'All',
                          observed = False,
                          sort = True)
answer_table1

sum                              
Months                                01    02    03    04    05    06
First Product Brand                                                   
Candy City | Gummy Worms            4844  4818  5178  4881  4946  5024
Candy City | Orange Beans           4947  5162  4783  4982  4999  5024
Exotic Extras | Beef Chicharon      4957  5099  4813  4898  4945  4844
Exotic Extras | Kimchi and Seaweed  4857  4881  5047  4872  4859  5047
HealthyKid 3+ | Gummy Vitamins      4853  5078  5087  4859  4978  4899
HealthyKid 3+ | Nutrional Milk      4855  4844  4757  4785  4856  4783
HealthyKid 3+ | Yummy Vegetables    5080  4930  4888  5041  4965  4971

In [45]:
answer_table2 = pd.pivot_table(table_1, index = ['Second Product Brand'],
                          columns = 'Months',
                          values = 'Second Product Count',
                          aggfunc = ['sum'],
                          fill_value=None,
                          margins = False,
                          dropna = True,
                          margins_name = 'All',
                          observed = False,
                          sort = True)
answer_table2

sum                              
Months                                01    02    03    04    05    06
Second Product Brand                                                  
Candy City | Gummy Worms            3201  3361  3174  3327  3228  3281
Candy City | Orange Beans           3343  3285  3232  3357  3351  3303
Exotic Extras | Beef Chicharon      3143  3323  3387  3287  3382  3353
Exotic Extras | Kimchi and Seaweed  3197  3342  3294  3446  3222  3398
HealthyKid 3+ | Gummy Vitamins      3116  3296  3337  3280  3204  3317
HealthyKid 3+ | Nutrional Milk      3301  3203  3466  3290  3383  3335
HealthyKid 3+ | Yummy Vegetables    3278  3662  3299  3227  3154  3220
No Item                                0     0     0     0     0     0

In [46]:
answer_table3 = pd.pivot_table(table_1, index = ['Third Product Brand'],
                          columns = 'Months',
                          values = 'Third Product Count',
                          aggfunc = ['sum'],
                          fill_value=None,
                          margins = False,
                          dropna = True,
                          margins_name = 'All',
                          observed = False,
                          sort = True)
answer_table3

sum                              
Months                                01    02    03    04    05    06
Third Product Brand                                                   
Candy City | Gummy Worms            1514  1817  1634  1835  1627  1629
Candy City | Orange Beans           1484  1590  1596  1575  1614  1779
Exotic Extras | Beef Chicharon      1565  1579  1616  1705  1701  1705
Exotic Extras | Kimchi and Seaweed  1622  1726  1626  1603  1692  1659
HealthyKid 3+ | Gummy Vitamins      1712  1606  1721  1703  1766  1764
HealthyKid 3+ | Nutrional Milk      1571  1644  1653  1711  1642  1649
HealthyKid 3+ | Yummy Vegetables    1601  1664  1709  1593  1616  1531
No Item                                0     0     0     0     0     0

In [47]:
#table 1: breakdown of the count of each item sold per month
final_answer = answer_table1 + answer_table2 + answer_table3
final_answer = final_answer.drop('No Item', axis = 0)
final_answer

sum                             \
Months                                  01       02       03       04   
Candy City | Gummy Worms            9559.0   9996.0   9986.0  10043.0   
Candy City | Orange Beans           9774.0  10037.0   9611.0   9914.0   
Exotic Extras | Beef Chicharon      9665.0  10001.0   9816.0   9890.0   
Exotic Extras | Kimchi and Seaweed  9676.0   9949.0   9967.0   9921.0   
HealthyKid 3+ | Gummy Vitamins      9681.0   9980.0  10145.0   9842.0   
HealthyKid 3+ | Nutrional Milk      9727.0   9691.0   9876.0   9786.0   
HealthyKid 3+ | Yummy Vegetables    9959.0  10256.0   9896.0   9861.0   

                                                      
Months                                   05       06  
Candy City | Gummy Worms             9801.0   9934.0  
Candy City | Orange Beans            9964.0  10106.0  
Exotic Extras | Beef Chicharon      10028.0   9902.0  
Exotic Extras | Kimchi and Seaweed   9773.0  10104.0  
HealthyKid 3+ | Gummy Vitamins       9948.0   9980.0  
HealthyKid 3+ | Nutrional Milk       9881.0   9767.0  
HealthyKid 3+ | Yummy Vegetables     9735.0   9722.0